# Learning to Rank Demo

## Overview

In this notebook, we will...

## Contents

1. [Overview](#Overview)
1. [Install ml4ir](#Install-ml4ir)
1. [Look at the Data](#Look-at-the-Data)
1. [Define the FeatureConfig](#Define-the-FeatureConfig)
1. [Using ml4ir as a script](#Using-ml4ir-as-a-script)
1. [Using ml4ir as a library](#Using-ml4ir-as-a-library)
    1. [Setup](#Setup)
    1. [Load the FeatureConfig](#Load-the-FeatureConfig)
    1. [Create a RelevanceDataset](#Create-a-RelevanceDataset)
    1. [Define an InteractionModel](#Define-an-InteractionModel)
    1. [Define losses, metrics and optimizer](#Define-losses,-metrics-and-optimizer)
    1. [Define the scoring function, or the Scorer](#Define-a-scoring-function,-or-the-Scorer)
    1. [Combine it all to create a RankingModel](#Combine-it-all-to-create-a-RankingModel)
    1. [Train your RankingModel](#Train-your-RankingModel)
    1. [Save the trained RankingModel](#Save-the-trained-RankingModel)
1. [Let's try some Transfer Learning](#Let's-try-some-Transfer-Learning)
    1. [What does ml4ir save?](#What-does-ml4ir-save?)
    1. [Using pre-trained character embeddings](#Using-pre-trained-character-embeddings)

## Install ml4ir

In [ ]:
!pip install ml4ir

## Look at the data

In [ ]:
df_train = file_io.read_df(os.path.join(DATA_DIR, "train", "file_0.csv"))
df_train.head(10)

## Define the FeatureConfig

| Feature          | dtype    | TFRecord Type | Usage                                    |
| ---------------- | -------- | ------------- | ---------------------------------------- |
| page_views_score | float    | Sequence      | as is                                    |
| quality_score    | float    | Sequence      | as is                                    |
| text_match_score | float    | Sequence      | as is                                    |
| query_text       | string   | Context       | character embeddings -> biLSTM encoding  |
| domain_name      | string   | Context       | VocabLookup -> categorical embedding     |

## Define the ModelConfig

In [30]:
print(open("configs/activate_2020/model_config.yaml").read())

architecture_key: dnn
layers:
  - type: dense
    name: first_dense
    units: 256
    activation: relu
  - type: dropout
    name: first_dropout
    rate: 0.3
  - type: dense
    name: second_dense
    units: 64
    activation: relu
  - type: dense
    name: final_dense
    units: 1
    activation: null



## Using ml4ir as a script

In [31]:
!python ../ml4ir/applications/ranking/pipeline.py \
--data_dir ../ml4ir/applications/ranking/tests/data/csv \
--feature_config configs/activate_2020/feature_config.yaml \
--model_config configs/activate_2020/model_config.yaml \
--data_format csv \
--execution_mode train_inference_evaluate \
--loss_key rank_one_listnet \
--num_epochs 3 \
--models_dir ../models/activate_2020 \
--logs_dir ../logs/activate_2020 \
--run_id activate_demo

INFO: 2020-09-10 20:41:28.265 
Logging initialized. Saving logs to : ../logs/activate_2020/activate_demo
INFO: 2020-09-10 20:41:28.265 
Run ID: activate_demo
DEBUG: 2020-09-10 20:41:28.266 
CLI args: 
{
    "data_dir": "../ml4ir/applications/ranking/tests/data/csv",
    "data_format": "csv",
    "tfrecord_type": "sequence_example",
    "feature_config": "configs/activate_2020/feature_config.yaml",
    "model_file": "",
    "model_config": "configs/activate_2020/model_config.yaml",
    "optimizer_key": "adam",
    "loss_key": "rank_one_listnet",
    "metrics_keys": "['MRR', 'ACR']",
    "monitor_metric": "new_MRR",
    "monitor_mode": "max",
    "num_epochs": 3,
    "batch_size": 128,
    "learning_rate": 0.01,
    "learning_rate_decay": 0.9,
    "learning_rate_decay_steps": 1000,
    "compute_intermediate_stats": true,
    "execution_mode": "train_inference_evaluate",
    "random_state": 123,
    "run_id": "activate_demo",
    "run_group": "general",
    "run_notes": "",
    "models_di

INFO: 2020-09-10 20:41:28.314 
Writing SequenceExample protobufs to : ../ml4ir/applications/ranking/tests/data/csv/tfrecord/train/file_0.tfrecord
ERROR: 2020-09-10 20:41:28.341 
!!! Error Training Model: !!!
'query_id'
Traceback (most recent call last):
  File "/Users/ashish.srinivasa/search_relevance/ml4ir/python/env/.ranking_venv3/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'mask'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/ashish.sriniv

## Using ml4ir as a library

<img src="images/model_framework.png" alt="ml4ir Architecture" style="width: 500px;" align="center"/>

### Setup

In [ ]:
MODEL_CONFIG_PATH = "configs/activate_2020/model_config.yaml"
FEATURE_CONFIG_PATH = "configs/activate_2020/feature_config.yaml"

DATA_DIR = "../ml4ir/applications/ranking/tests/data/csv"
MODELS_DIR = '../models/activate_2020'
LOGS_DIR = '../logs/activate_2020'

MAX_SEQUENCE_SIZE = 25

In [ ]:
import logging
import tensorflow as tf
import os

from tensorflow.keras.metrics import Metric
from tensorflow.keras.optimizers import Optimizer
from typing import List, Union, Type

from ml4ir.base.io.local_io import LocalIO
from ml4ir.base.io.file_io import FileIO
from ml4ir.base.config.keys import *
from ml4ir.base.data.relevance_dataset import RelevanceDataset
from ml4ir.base.features.feature_config import FeatureConfig, SequenceExampleFeatureConfig
from ml4ir.base.model.scoring.scoring_model import RelevanceScorer
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.model.scoring.interaction_model import InteractionModel, UnivariateInteractionModel
from ml4ir.base.model.losses.loss_base import RelevanceLossBase
from ml4ir.base.model.optimizer import get_optimizer
from ml4ir.applications.ranking.model.ranking_model import RankingModel
from ml4ir.applications.ranking.config.keys import LossKey, MetricKey, ScoringTypeKey
from ml4ir.applications.ranking.model.losses import loss_factory
from ml4ir.applications.ranking.model.metrics import metric_factory

In [ ]:
# Set up file I/O handler
file_io : FileIO = LocalIO()
    
# Create directories for models and logs
file_io.make_directory(LOGS_DIR, clear_dir=True)
file_io.make_directory(MODELS_DIR, clear_dir=True)

# Set up logger
logger = logging.getLogger()

### Load the FeatureConfig

In [ ]:
feature_config: SequenceExampleFeatureConfig = FeatureConfig.get_instance(
    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
    feature_config_dict=file_io.read_yaml(FEATURE_CONFIG_PATH),
    logger=logger)
print("Training features\n-----------------")
print("\n".join(feature_config.get_train_features(key="name")))

### Create a RelevanceDataset

In [ ]:
def get_relevance_dataset(preprocessing_keys_to_fns={}):

    return RelevanceDataset(data_dir=DATA_DIR,
                            data_format=DataFormatKey.TFRECORD,
                            feature_config=feature_config,
                            tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                            max_sequence_size=MAX_SEQUENCE_SIZE,
                            batch_size=128,
                            preprocessing_keys_to_fns=preprocessing_keys_to_fns,
                            file_io=file_io,
                            logger=logger)

ranking_dataset = get_relevance_dataset()

### Define an InteractionModel

In [ ]:
interaction_model: InteractionModel = UnivariateInteractionModel(
                                            feature_config=feature_config,
                                            tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                            max_sequence_size=MAX_SEQUENCE_SIZE,
                                            file_io=file_io,
                                        )

### Define losses, metrics and optimizer
##### Use predefined losses, metrics and optimizers or create your own!

In [ ]:
# Define loss object from loss key
loss: RelevanceLossBase = loss_factory.get_loss(
                                loss_key=LossKey.RANK_ONE_LISTNET,
                                scoring_type=ScoringTypeKey.POINTWISE)
    
# Define metrics objects from metrics keys
metrics: List[Union[Type[Metric], str]] = [metric_factory.get_metric(metric_key="MRR")]
    
# Define optimizer
optimizer: Optimizer = get_optimizer(
                            optimizer_key=OptimizerKey.ADAM,
                            learning_rate=0.001
                        )

### Define a scoring function, or the Scorer

In [ ]:
scorer: RelevanceScorer = RelevanceScorer.from_model_config_file(
    model_config_file=MODEL_CONFIG_PATH,
    interaction_model=interaction_model,
    loss=loss,
    logger=logger,
    file_io=file_io,
)

### Combine it all to create a RankingModel

In [ ]:
ranking_model: RelevanceModel = RankingModel(
                                    feature_config=feature_config,
                                    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                    scorer=scorer,
                                    metrics=metrics,
                                    optimizer=optimizer,
                                    file_io=file_io,
                                    logger=logger,
                                )

### Train your RankingModel

In [ ]:
ranking_model.fit(dataset=ranking_dataset,
                  num_epochs=3, 
                  models_dir=MODELS_DIR,
                  logs_dir=LOGS_DIR,
                  monitor_metric="new_MRR",
                  monitor_mode="max")

## Let's try some Transfer Learning

### What does ml4ir save?

### Using pre-trained character embeddings

In [ ]:
ranking_model.model.get_layer("query_text_bytes_embedding").get_weights()

In [ ]:
initialize_layers_dict = {
    "query_text_bytes_embedding" : "../models/test_wandb2/final/layers/query_text_bytes_embedding.npy"
}
freeze_layers_list = ["query_text_bytes_embedding"]
ranking_model: RelevanceModel = RankingModel(
                                    feature_config=feature_config,
                                    tfrecord_type=TFRecordTypeKey.SEQUENCE_EXAMPLE,
                                    scorer=scorer,
                                    metrics=metrics,
                                    optimizer=optimizer,
                                    initialize_layers_dict=initialize_layers_dict,
                                    freeze_layers_list=freeze_layers_list,
                                    file_io=file_io,
                                    logger=logger,
                                )

In [ ]:
ranking_model.model.get_layer("query_text_bytes_embedding").get_weights()